<h1>Matrix Factorization for Check-In Prediction</h1>
<h2>MS&E 234</h2>

<h3>Setup</h3>

We import necessary libraries and read in our dataset.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

The check-in dataset below is modified from the original check-in dataset to 1) limit data points to users with known genders and 2) include venue category.

In [2]:
check_ins = pd.read_csv('gender-checkins-complete.csv', delimiter='\t', index_col=0)

In [3]:
check_ins.head()

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
0,21939,4dd53b151f6ec4e0bb8c0480,Tue Apr 03 18:00:49 +0000 2012,-240,39.285600,-76.612047,Clothing Store,US
1,163646,4b70040ff964a52080032de3,Tue Apr 03 18:01:31 +0000 2012,-240,25.716845,-80.281378,College Cafeteria,US
2,256534,4b29929cf964a5200fa124e3,Tue Apr 03 18:01:37 +0000 2012,-360,40.726135,-111.852087,American Restaurant,US
3,176836,4b66f88ff964a520eb322be3,Tue Apr 03 18:01:40 +0000 2012,-300,29.661129,-95.115077,Community College,US
4,181560,4bc7086715a7ef3bef9878da,Tue Apr 03 18:02:41 +0000 2012,-240,40.745164,-73.982519,Medical Center,US


We will choose the category to predict: specific venue or venue category.

In [4]:
category = 'Category'  # 'Venue_ID'

### Preprocessing

The following functions reduce the data to users who have made 80+ check-ins in the past.

In [5]:
threshold = 80 # Any user that has less than (threshold) check-ins will be removed.

value_counts = check_ins['User_ID'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= threshold].index
check_ins = check_ins[~check_ins['User_ID'].isin(to_remove)]

In [6]:
"There are %d check-ins, made by %d users." %(len(check_ins), check_ins['User_ID'].nunique())

'There are 465613 check-ins, made by 2290 users.'

In [7]:
check_ins[category].nunique()

433

### Data Split

In [8]:
split = 0.8

X_train = (check_ins.groupby('User_ID', group_keys=False).apply(lambda x: x.nlargest(int(len(x) * split), 'User_ID')))
X_train.head()

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
8621,54,4bb3a86c4019a593e14138b8,Sat Apr 07 23:51:17 +0000 2012,-600,20.648591,-156.442308,Surf Spot,US
14826,54,4c3bf087b36ac928e4850386,Mon Apr 09 23:58:24 +0000 2012,-600,21.008267,-156.556955,Lake,US
21779,54,4c0f13fcd64c0f47b055295d,Thu Apr 12 07:16:01 +0000 2012,-600,20.926547,-156.694711,Japanese Restaurant,US
28025,54,4de0117c45dd3eae8764d6ac,Fri Apr 13 22:48:37 +0000 2012,-420,37.781213,-122.402973,Tech Startup,US
28446,54,49ca8f4df964a520b9581fe3,Sat Apr 14 00:23:43 +0000 2012,-420,37.782464,-122.407823,Coffee Shop,US


In [9]:
X_test = check_ins[~check_ins.isin(X_train)].dropna()

### Ratings Matrix Creation

In [10]:
R_df = X_train.groupby(['User_ID', category]).size().reset_index(name="Frequency")

In [11]:
Total_Visits = X_train.groupby(['User_ID']).size().reset_index(name="Total_Visits")
R_df = pd.merge(R_df, Total_Visits, on = 'User_ID', how='left', sort = 'False')

In [12]:
np.mean(Total_Visits['Total_Visits'])

162.24366812227075

In [13]:
R_df['Adj_Freq'] = R_df['Frequency'] / R_df['Total_Visits'] * 1.0

In [14]:
R_df.head()

,User_ID,Category,Frequency,Total_Visits,Adj_Freq
0,54,Airport,5,108,0.046296
1,54,American Restaurant,1,108,0.009259
2,54,Bar,1,108,0.009259
3,54,Beach,2,108,0.018519
4,54,Breakfast Spot,1,108,0.009259


In [15]:
R_df = R_df.pivot(index='User_ID', columns=category, values='Adj_Freq').fillna(0)

In [16]:
R_df.head()

Category,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Airport Tram,American Restaurant,...,Well,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yogurt,Zoo
User_ID,,,,,,,,,,,,,,,,,,,,,
54,0.0,0.0,0.0,0.046296,0.0,0.0,0.0,0.000000,0.0,0.009259,...,0.0,0.0,0.0,0.0,0.009259,0.0,0.0,0.0,0.0,0.0000
182,0.0,0.0,0.0,0.012500,0.0,0.0,0.0,0.012500,0.0,0.012500,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0125
346,0.0,0.0,0.0,0.010526,0.0,0.0,0.0,0.000000,0.0,0.015789,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000
419,0.0,0.0,0.0,0.014706,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000
541,0.0,0.0,0.0,0.072289,0.0,0.0,0.0,0.006024,0.0,0.030120,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000


In [17]:
R = R_df.as_matrix()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


### Singular Value Decomposition

In [18]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(R, k = 20)  # Can replace k with different values.

In [19]:
sigma = np.diag(sigma)

In [20]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [21]:
preds_df.head()

Category,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Airport Tram,American Restaurant,...,Well,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yogurt,Zoo
0,0.000063,-0.000003,0.000139,0.045503,-0.000004,0.002126,0.000859,0.004490,0.000084,0.008494,...,0.000900,0.000271,0.001409,0.000326,0.000581,0.000830,-0.000005,0.000596,-0.000002,0.000483
1,0.000385,0.000006,0.000109,0.010712,0.000030,0.001463,0.000385,0.004658,0.000082,0.019536,...,0.000032,0.000300,0.000721,0.000395,0.000467,0.001639,0.000437,0.000445,0.000019,0.001318
2,0.000045,-0.000004,0.000029,0.009797,-0.000005,0.001050,0.000163,0.002933,0.000013,0.000097,...,-0.000030,-0.000010,0.000089,0.000662,0.000719,0.002182,0.000465,-0.000097,-0.000018,0.000388
3,-0.000018,-0.000005,0.000344,0.015488,0.000011,-0.000166,-0.000948,0.002133,-0.000084,-0.001570,...,0.000475,0.000015,0.001440,0.000081,0.000443,0.000352,0.001000,0.000672,0.000031,0.000534
4,0.000027,-0.000008,0.000020,0.071704,0.000015,0.003600,0.002492,0.008982,0.000217,0.015151,...,-0.000037,0.000207,0.001237,0.000397,0.001248,0.002755,0.000111,0.000437,-0.000005,0.000665


In [22]:
def predict_check_ins(preds_df, num_recommendations=25):
    # Mask predictions that were already visited.
    mask = R_df.reset_index().drop(columns=['User_ID']) != 0
    preds = preds_df.mask(mask, other=float('-inf'))
    correct = 0
    # Make predictions.
    for row in preds.index:
        preds_row = preds.iloc[row].sort_values(ascending=False)[0:num_recommendations]
        X_test_rows = X_test[X_test['User_ID'] == R_df.index[row]]
        X_test_rows = X_test_rows[X_test_rows[category].isin(preds_row.index)]
        if not X_test_rows.empty: correct += 1
    accuracy = correct / len(preds.index)
    return accuracy

acc = predict_check_ins(preds_df)

In [23]:
acc

0.8436681222707424

### Analysis